In this assignment, you will use a pre-trained convnet to produce features for a classifier that can detect a single object type. This notebook has some code to help you get started. 

In [1]:
from imutils import paths
import pandas as pd
import requests
import cv2
import os
from os import listdir
from os.path import isfile, join
import os.path as osp
from tqdm import tqdm_notebook as tqdm
from google_images_download import google_images_download

img_folder = '../downloads'

In [2]:
def build_arguments(word):
    args = {}
    args['keywords'] = word
    args['limit'] = 100
    args['format'] = 'png'
    #args['usage_rights'] = 'labeled-for-nocommercial-reuse'
    return args

response = google_images_download.googleimagesdownload()

### Run the following two cells

- change the positive and negative words
- running each cell downloads 100 images into a `downloads` folder with a subfolder of your word
- `word` can be any string or search criteria

In [3]:
# positive examples

word = 'University logo'
absolute_image_paths = response.download(build_arguments(word))


Item no.: 1 --> Item name = University logo
Evaluating...
Starting Download...
Completed Image ====> 1. 1280px-athabasca_university_logo_2017.svg.png
Completed Image ====> 2. vert2_blue_seal_wilkes.png
Completed Image ====> 3. nmu_stack_transparent.png
Completed Image ====> 4. university.logo_.small_.vertical.white_.png
Completed Image ====> 5. university_of_oxford-logo-2acbb1aa61-seeklogo.com.png
Completed Image ====> 6. university_logo_small_vertical_blue.png
Completed Image ====> 7. towsonu-logo.png
Completed Image ====> 8. filetodownload,200788,en.png
Completed Image ====> 9. ?media_id=109914737339.jpg
Completed Image ====> 10. cincinnati_christian_university_logo_detail.png
Completed Image ====> 11. university_logos_buildings.png
Completed Image ====> 12. gu-logo-stacked.png
Completed Image ====> 13. nccu_vert_color_logo.png
Completed Image ====> 14. logorev-1.png
Completed Image ====> 15. ashford_university_full_color_logo.png
Completed Image ====> 16. academic-donts2.png
Comple

In [5]:
# negative examples

word = 'random'
absolute_image_paths = response.download(build_arguments(word))


Item no.: 1 --> Item name = random
Evaluating...
Looks like we cannot locate the path the 'chromedriver' (use the '--chromedriver' argument to specify the path to the executable.) or google chrome browser is not installed on your machine (exception: argument of type 'NoneType' is not iterable)


SystemExit: 

## 1.) Run the following cell

* This imports needed Keras libraries
* Then, it gets the trained VGG19 imagenet model
* Then, it prints out the names of all the layers in that model

In [74]:
import numpy as np
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model

base_model = VGG19(weights='imagenet')

for layer in base_model.layers:
    print(layer.name)

input_6
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_conv4
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_conv4
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_conv4
block5_pool
flatten
fc1
fc2
predictions


### 2.) Determine your output layer

- try `predictions` first

In [103]:
layer = 'predictions'

model = Model(input=base_model.input, output=base_model.get_layer(layer).output)

C:\Users\Mayson\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  This is separate from the ipykernel package so we can avoid doing imports until


### Run the following cell

In [76]:
def get_image(img_path, xs=224,ys=224):
    x = image.load_img(img_path, target_size=(xs, ys))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    return x

def get_img_features(model, img):
    img = preprocess_input(img)
    yhat = model.predict(img)
    return yhat

def get_image_features(word):
    files = [f for f in listdir(osp.join(img_folder, word))] # grab all of the images in the folder
    image_vectors = []
    for f in tqdm(files):
        img = get_image(osp.join(img_folder, word, f)) 
        x_feats = get_img_features(model, img).flatten() # get features for each image
        image_vectors.append(x_feats) 
    return np.array(image_vectors)

## 3.) Evaluate a classifier for your `word`

* Using the positive and negative output from `base_model`, train a classifier (it can be a linear classifier from scikit-learn, if you'd like, but I would recommend the Keras Dense network we built for the previous assignment). 
* You'll need to split your data into Train and Test (I would recommend using half of the data for training, half for testing; you may opt for downloading more positive and negative examples)

In [39]:
pos_images = get_image_features('red') # get positive image vectors
neg_images = get_image_features('random') # get negative image vectors

### Prepare the data. Split to train/test sets

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
pos_images = pd.DataFrame(pos_images)
neg_images = pd.DataFrame(neg_images)
# Set labels for each set of data
pos_images['label'] = 1
neg_images['label'] = 0

# Combine the data
images = pos_images.append(neg_images, ignore_index=True)

# Shuffle the data 
images = images.sample(frac=1).reset_index(drop=True)

# Drop the labels
y = images['label']
images.drop(columns=["label"], inplace=True)

#split into train and test
X_train, X_test, y_train, y_test = train_test_split(images, y)

In [79]:
images[:5]

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,6.301458e-09,5.907184e-06,8.037114e-07,1.626123e-06,1.852689e-07,1.096791e-07,1.542192e-07,0.000006,1.241482e-06,2.855139e-06,...,6.277209e-07,0.000095,1.048783e-06,2.340776e-08,5.418072e-07,0.000005,0.000008,4.838635e-08,0.000002,0.000073
1,1.135017e-08,1.531346e-05,4.887981e-07,2.511569e-07,4.812061e-07,2.258283e-06,4.827080e-08,0.000001,2.384241e-07,5.345002e-06,...,9.852550e-06,0.000004,6.402079e-06,1.458465e-08,6.983396e-07,0.000002,0.000004,1.067255e-06,0.000004,0.000035
2,1.191789e-04,1.543166e-03,5.016750e-04,5.494332e-04,1.478872e-03,6.702818e-04,2.337048e-04,0.000241,9.305143e-05,3.954895e-04,...,5.605357e-04,0.000431,1.631531e-04,6.200447e-05,3.375948e-04,0.000373,0.000118,1.471096e-04,0.000480,0.001219
3,4.082322e-07,6.750598e-07,5.681961e-07,4.025774e-06,3.143018e-06,3.651271e-06,1.250586e-05,0.000004,4.498328e-06,7.097250e-07,...,2.134363e-06,0.000005,4.505874e-07,1.340546e-05,1.052134e-05,0.000003,0.000065,2.912846e-06,0.000090,0.004610
4,7.473658e-08,5.709233e-05,1.214775e-06,2.123569e-06,2.062873e-06,2.842061e-06,1.706382e-07,0.000001,1.056008e-06,1.879740e-05,...,3.620785e-05,0.000011,3.839684e-06,9.268224e-08,1.594954e-06,0.000007,0.000006,1.191560e-06,0.000022,0.000062


### Define model, train

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [81]:
lr = LogisticRegression()
model = lr.fit(X_train, y_train)

C:\Users\Mayson\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Evaluate

In [82]:
y_prediction = model.predict(X_test)

In [83]:
accuracy_score(y_test, y_prediction)

0.6376811594202898

## 4.) Try a Different `base_model` 

* Repeat steps 3 and 4 above, only this time use a [pre-trained model other than VGG19](https://keras.io/applications/)
* (Answer in a markdown cell): Which model+layer works the best for this data? Why do you think that is?

In [95]:
from keras.applications.xception import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input

In [102]:
base_model = Xception(weights='imagenet')

In [106]:
layer = 'block4_pool'

model = Model(input=base_model.input, output=base_model.get_layer(layer).output)

C:\Users\Mayson\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [107]:
pos_images = get_image_features('red') # get positive image vectors
neg_images = get_image_features('random') # get negative image vectors

C:\Users\Mayson\Anaconda3\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


In [108]:
pos_images = pd.DataFrame(pos_images)
neg_images = pd.DataFrame(neg_images)
# Set labels for each set of data
pos_images['label'] = 1
neg_images['label'] = 0

# Combine the data
images = pos_images.append(neg_images, ignore_index=True)

# Shuffle the data 
images = images.sample(frac=1).reset_index(drop=True)

# Drop the labels
y = images['label']
images.drop(columns=["label"], inplace=True)

#split into train and test
X_train, X_test, y_train, y_test = train_test_split(images, y)

In [109]:
lr2 = LogisticRegression()
model = lr2.fit(X_train, y_train)

C:\Users\Mayson\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [111]:
y_prediction2 = model.predict(X_test)

In [112]:
accuracy_score(y_test, y_prediction2)

0.8405797101449275

The block4_pool layer worked better for me than the predictions layer. I believe that is because earlier layers are more generic than later layers

In [113]:
from client.api.notebook import Notebook
ok = Notebook('a7.ok')
ok.auth(inline=True)

Assignment: A7 Transfer Learning
OK, version v1.13.11



ERROR  | auth.py:91 | {'error': 'invalid_grant'}



Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: bYZYyUQk1Q3uxRkuacqSC3PGM8VHPT
Successfully logged in as MaysonGreen@u.boisestate.edu


In [114]:
ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'A7-transfer-learning.ipynb'.
Submit... 100% complete
Submission successful for user: MaysonGreen@u.boisestate.edu
URL: https://okpy.org/boisestate/cs4-533/sp19/a7/submissions/k8MzgE

